

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [19]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [52]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size = hidden_size
    self.linear_z = torch.nn.Linear(27 + hidden_size, hidden_size)
    self.linear_r = torch.nn.Linear(27 + hidden_size, hidden_size)
    self.linear_h = torch.nn.Linear(27 + hidden_size, hidden_size)
    self.linear = torch.nn.Linear(hidden_size, 27)
    self.softmax = torch.nn.Softmax(dim=1)

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    batch_size, seq_length, _ = x.shape
    h = torch.zeros(batch_size, self.hidden_size).to(x.device)
    out = torch.zeros(batch_size, seq_length, self.hidden_size).to(x.device)
    for t in range(seq_length):
        x_t = x[:, t, :]
        z_t = torch.sigmoid(self.linear_z(torch.cat([x_t, h], dim=1)))
        r_t = torch.sigmoid(self.linear_r(torch.cat([x_t, h], dim=1))) 
        r_temp = r_t * h
        h_temp = torch.tanh(self.linear_h(torch.cat([x_t, r_temp], dim=1)))
        h = (1 - z_t) * h + z_t * h_temp
        out[:, t, :] = h

    out = self.linear(out)
    out = self.softmax(out)
    return out

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    s_onehot = torch.zeros(1, len(s), 27)
    for i in range(len(s)):
        if s[i] == ' ':
          s_onehot[0, i, 0] = 1
        else:
          s_onehot[0, i, ord(s[i]) - ord('a') + 1] = 1

    output = self.forward(s_onehot)

    # Convert to string
    s_out = ''
    for i in range(output.size(1)):
        c_out = torch.argmax(output[0, i]).item()
        if c_out == 0:
            s_out += ' '
        else:
            s_out += chr(c_out + ord('a') - 1)

    return s_out

## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [28]:
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  D = DELAY
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D
    break

  return correct / total

In [55]:
import time
start_time = time.time()

# TODO
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRUMemory(100).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_loader = torch.utils.data.DataLoader(ds, batch_size = 100)

for epoch in range(3):
  print("Epoch", epoch+1)

  for batch_idx, (data, res) in enumerate(train_loader):
    optimizer.zero_grad()
    data = idx_to_onehot(data, k=N+1).to(device)
    res = res.to(device)
    pred = model(data)
    loss = criterion(pred.view(-1, N + 1), res.view(-1))
    loss.backward()
    optimizer.step()
    if batch_idx % 100 == 0:
      print("Batch", batch_idx, "Loss:", loss.item())

model = model.cpu()
end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Epoch 1
Batch 0 Loss: 3.2958171367645264
Batch 100 Loss: 3.2303214073181152
Batch 200 Loss: 3.2299904823303223
Batch 300 Loss: 3.2298343181610107
Batch 400 Loss: 3.2295117378234863
Batch 500 Loss: 3.2166941165924072
Batch 600 Loss: 3.2033050060272217
Batch 700 Loss: 3.168661117553711
Batch 800 Loss: 3.1118245124816895
Batch 900 Loss: 3.0740621089935303
Batch 1000 Loss: 2.9269354343414307
Batch 1100 Loss: 2.811577081680298
Batch 1200 Loss: 2.758138418197632
Batch 1300 Loss: 2.730489730834961
Batch 1400 Loss: 2.719351053237915
Batch 1500 Loss: 2.7109925746917725
Batch 1600 Loss: 2.711327075958252
Batch 1700 Loss: 2.7301177978515625
Batch 1800 Loss: 2.704587936401367
Batch 1900 Loss: 2.7058117389678955
Epoch 2
Batch 0 Loss: 2.7232260704040527
Batch 100 Loss: 2.7246673107147217
Batch 200 Loss: 2.7077648639678955
Batch 300 Loss: 2.7203781604766846
Batch 400 Loss: 2.7080981731414795
Batch 500 Loss: 2.6987626552581787
Batch 600 Loss: 2.717864990234375
Batch 700 Loss: 2.707404375076294
Batch 8

## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [57]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [80]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.max_delay = max_delay
    self.hidden_size = hidden_size
    self.linear_z = torch.nn.Linear(27 + hidden_size, hidden_size)
    self.linear_r = torch.nn.Linear(27 + hidden_size, hidden_size)
    self.linear_h = torch.nn.Linear(27 + hidden_size, hidden_size)
    self.linear = torch.nn.Linear(hidden_size, 27)
    self.softmax = torch.nn.Softmax(dim=1)
    self.fc = torch.nn.Linear(hidden_size + hidden_size, hidden_size)

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

    # TODO
    batch_size, seq_length, _ = x.shape
    h = torch.zeros(batch_size, self.hidden_size)
    delay_buffer = torch.zeros(batch_size, self.max_delay + 1, self.hidden_size)
    out = torch.zeros(batch_size, seq_length, self.hidden_size).to(x.device)

    for t in range(seq_length):
        z_t = torch.sigmoid(self.linear_z(torch.cat([x[:, t, :], h], dim=1)))
        r_t = torch.sigmoid(self.linear_r(torch.cat([x[:, t, :], h], dim=1)))
        r_temp = r_t * h 
        h_temp = torch.tanh(self.linear_h(torch.cat([x[:, t, :], r_temp], dim=1)))
        h = (1 - z_t) * h + z_t * h_temp

        # Get h_delay
        delay_buffer = torch.cat([h.unsqueeze(dim=1), delay_buffer[:, :-1]], dim=1)
        delay_pos = delays if isinstance(delays, torch.Tensor) else torch.tensor([delays])
        h_delay = delay_buffer[torch.arange(batch_size), delay_pos.long()]

        # Combine and fc layer
        h_combined = torch.cat([h, h_delay], dim=1)
        h_t = self.fc(h_combined)
        out[:, t, :] = h_delay

    out = self.linear(out)
    out = self.softmax(out)
    return out
    

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    s_onehot = torch.zeros(1, len(s), 27)
    for i in range(len(s)):
        if s[i] == ' ':
          s_onehot[0, i, 0] = 1
        else:
          s_onehot[0, i, ord(s[i]) - ord('a') + 1] = 1

    delay_input = torch.zeros(1)
    delay_input = delay
    output = self.forward(s_onehot, delay_input)

    # Convert to string
    s_out = ''
    for i in range(output.size(1)):
        c_out = torch.argmax(output[0, i]).item()
        if c_out == 0:
            s_out += ' '
        else:
            s_out += chr(c_out + ord('a') - 1)

    return s_out


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [62]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d
  return correct / total

In [82]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.
device = torch.device('cpu')
ds = VariableDelayEchoDataset(max_delay=MAX_DELAY, seq_length=SEQ_LENGTH, size=DATASET_SIZE)
model = VariableDelayGRUMemory(80, MAX_DELAY).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_loader = torch.utils.data.DataLoader(ds, batch_size = 64)

for epoch in range(1):
  print("Epoch", epoch+1)

  for batch_idx, (data, delay, res) in enumerate(train_loader):
    optimizer.zero_grad()
    data = idx_to_onehot(data, k=N+1).to(device)
    res = res.to(device)
    pred = model(data, delay)
    loss = criterion(pred.view(-1, N + 1), res.view(-1))
    loss.backward()
    optimizer.step()
    if batch_idx % 100 == 0:
      print("Batch", batch_idx, "Loss:", loss.item())

end_time = time.time()
model = model.cpu()

assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

Epoch 1
Batch 0 Loss: 3.2961738109588623
Batch 100 Loss: 3.2091262340545654
Batch 200 Loss: 2.7696821689605713
Batch 300 Loss: 2.7153637409210205
Batch 400 Loss: 2.712484121322632
Batch 500 Loss: 2.7040889263153076
Batch 600 Loss: 2.7180933952331543
Batch 700 Loss: 2.726637125015259
Batch 800 Loss: 2.7233896255493164
Batch 900 Loss: 2.6904845237731934
Batch 1000 Loss: 2.709113597869873
Batch 1100 Loss: 2.714386224746704
Batch 1200 Loss: 2.706068515777588
Batch 1300 Loss: 2.7353627681732178
Batch 1400 Loss: 2.7065770626068115
Batch 1500 Loss: 2.708585023880005
Batch 1600 Loss: 2.731459379196167
Batch 1700 Loss: 2.701019048690796
Batch 1800 Loss: 2.692622423171997
Batch 1900 Loss: 2.72145676612854
Batch 2000 Loss: 2.705446243286133
Batch 2100 Loss: 2.7000861167907715
Batch 2200 Loss: 2.7161219120025635
Batch 2300 Loss: 2.710541248321533
Batch 2400 Loss: 2.728123903274536
Batch 2500 Loss: 2.711383104324341
Batch 2600 Loss: 2.687221050262451
Batch 2700 Loss: 2.7075867652893066
Batch 2800 L